In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Dense
from keras.regularizers import l2
from tensorflow import keras

In [4]:
lastframe_data= pd.read_csv("../Data/glm_groups.csv")
lastframe_data = lastframe_data.drop(['Unnamed: 0.1','Unnamed: 0', 'rankc','id','game_clock'], axis=1)

In [6]:
def maximum_absolute_scaling(df):
    # copy the dataframe
    df_scaled = df.copy()
    # apply maximum absolute scaling
    for column in df_scaled.columns:
        df_scaled[column] = df_scaled[column]  / df_scaled[column].abs().max()
    return df_scaled
    
# call the maximum_absolute_scaling function
DF_features = maximum_absolute_scaling(lastframe_data)
DF_features = DF_features.drop(columns=['EVENTMSGTYPE'])

In [7]:
y_pred = lastframe_data['EVENTMSGTYPE'].map({2:1,1:0})

In [9]:
model = Sequential()
model.add(Dense(1024, input_dim=3, activation=tf.nn.relu))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(512, activation=tf.nn.relu))
model.add(BatchNormalization())
model.add(Dense(256,activation = tf.nn.relu, kernel_regularizer=keras.regularizers.l2(l=0.1)))
model.add(BatchNormalization())
model.add(Dense(128,activation = tf.nn.relu))
model.add(Dense(1, activation=tf.nn.sigmoid))

In [10]:
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [11]:
tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=0, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(
DF_features, y_pred, test_size=0.1)

In [13]:
x_train, x_val, y_train, y_val = train_test_split(
x_train, y_train, test_size=0.33)



In [14]:
model.fit(x_train, y_train,validation_data = (x_val,y_val), callbacks=[callback], epochs=50, batch_size=64)

Epoch 1/50
195/195 [==============================] - 4s 16ms/step - loss: 13.2712 - accuracy: 0.6519 - val_loss: 0.9886 - val_accuracy: 0.4053
Epoch 2/50
195/195 [==============================] - 2s 11ms/step - loss: 0.6182 - accuracy: 0.7972 - val_loss: 0.6796 - val_accuracy: 0.6863
Epoch 3/50
195/195 [==============================] - 2s 11ms/step - loss: 0.5122 - accuracy: 0.8103 - val_loss: 0.6189 - val_accuracy: 0.5996
Epoch 4/50
195/195 [==============================] - 2s 12ms/step - loss: 0.5197 - accuracy: 0.7785 - val_loss: 0.6063 - val_accuracy: 0.7289
Epoch 5/50
195/195 [==============================] - 2s 12ms/step - loss: 0.5553 - accuracy: 0.7437 - val_loss: 0.5646 - val_accuracy: 0.6453
Epoch 6/50
195/195 [==============================] - 2s 13ms/step - loss: 0.5112 - accuracy: 0.7717 - val_loss: 0.4980 - val_accuracy: 0.8432
Epoch 7/50
195/195 [==============================] - 2s 13ms/step - loss: 0.5417 - accuracy: 0.7471 - val_loss: 0.5103 - val_accuracy: 0.814

In [15]:
_, accuracy = model.evaluate(x_test, y_test)

65/65 [==============================] - 0s 2ms/step - loss: 0.6047 - accuracy: 0.7087


In [119]:
model.predict(x_train)

array([[0.80371857],
       [0.98154676],
       [0.69264686],
       ...,
       [0.40631032],
       [0.42610464],
       [0.821939  ]], dtype=float32)

In [16]:
model_name = 'nba_app'
model_version ='0001'
model_path = os.path.join('../',model_name,model_version) 
tf.saved_model.save(model,model_path)

INFO:tensorflow:Assets written to: ../nba_app/0001/assets


In [17]:
! pip install google-api-python-client

     |████████████████████████████████| 7.1 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 95 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 198 kB 5.7 MB/s eta 0:00:01
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [18]:
! pip install --upgrade google-api-python-client oauth2client

     |████████████████████████████████| 98 kB 4.1 MB/s eta 0:00:011
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [20]:
! export GOOGLE_APPLICATION_CREDENTIALS='/Users/johnzhang/Downloads/nifty-buffer-312520-8712e04ee4f8.json'

In [21]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ='/Users/johnzhang/Downloads/nifty-buffer-312520-8712e04ee4f8.json'

In [50]:
import googleapiclient.discovery
from google.api_core.client_options import ClientOptions


endpoint = 'https://ml.googleapis.com'
client_options = ClientOptions(api_endpoint=endpoint)
# region could matter 
# Under NaviMenu > AI Platform > Models
# my "Endpoint" is us-east4-ml

# this finds the model_path
ml_resource = googleapiclient.discovery.build("ml", "v1", client_options=client_options, cache_discovery=False).projects()
# https://stackoverflow.com/questions/40154672/importerror-file-cache-is-unavailable-when-using-python-client-for-google-ser
client_options

ClientOptions: {'api_endpoint': 'https://ml.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None}

In [89]:
data = x_train.to_numpy()
neural_data = data.tolist()

In [131]:
import json

project_id = "nifty-buffer-312520" # change this to your project ID
model_id = "nba_app_deployment" # named in GCP
model_ver = "version_1" # named in GCP
model_path = "projects/{}/models/{}/versions/{}".format(project_id, model_id, model_ver)
print(model_path)

request = ml_resource.predict(name=model_path, body={"instances":neural_data})
response = request.execute()    

projects/nifty-buffer-312520/models/nba_app_deployment/versions/version_1


In [132]:
response

{'predictions': [{'dense_4': [0.8037185668945312]},
  {'dense_4': [0.9815467596054077]},
  {'dense_4': [0.6926466822624207]},
  {'dense_4': [0.37046748399734497]},
  {'dense_4': [0.43375009298324585]},
  {'dense_4': [0.4483766257762909]},
  {'dense_4': [0.9780006408691406]},
  {'dense_4': [0.44047749042510986]},
  {'dense_4': [0.35197728872299194]},
  {'dense_4': [0.3002115786075592]},
  {'dense_4': [0.4755113124847412]},
  {'dense_4': [0.4755113124847412]},
  {'dense_4': [0.4155969023704529]},
  {'dense_4': [0.515251100063324]},
  {'dense_4': [0.4092586636543274]},
  {'dense_4': [0.35833385586738586]},
  {'dense_4': [0.45552563667297363]},
  {'dense_4': [0.3920500874519348]},
  {'dense_4': [0.932402491569519]},
  {'dense_4': [0.9553180932998657]},
  {'dense_4': [0.3886689841747284]},
  {'dense_4': [0.9900475144386292]},
  {'dense_4': [0.4755113124847412]},
  {'dense_4': [0.4755113124847412]},
  {'dense_4': [0.4755113124847412]},
  {'dense_4': [0.9899101257324219]},
  {'dense_4': [0.95

In [75]:
x_train[:1]

,x,y,z
4245,0.160576,0.633854,0.609896


In [60]:
y_train

4245     1
19732    1
1089     1
14331    1
6635     0
        ..
10562    1
7930     1
9553     1
14065    0
11811    1
Name: EVENTMSGTYPE, Length: 12478, dtype: int64